# Real-world use-cases at scale!



# Imports

Let's start with imports.

In [ ]:
import sys
sys.path.append("gpu_bdb_runner.egg")

In [ ]:
import gpu_bdb_runner as gpubdb
import os
import inspect
from highlight_code import print_code

In [ ]:
config_options = {}
config_options['JOIN_PARTITION_SIZE_THRESHOLD'] = os.environ.get("JOIN_PARTITION_SIZE_THRESHOLD", 300000000)
config_options['MAX_DATA_LOAD_CONCAT_CACHE_BYTE_SIZE'] =  os.environ.get("MAX_DATA_LOAD_CONCAT_CACHE_BYTE_SIZE", 400000000)
config_options['BLAZING_DEVICE_MEM_CONSUMPTION_THRESHOLD'] = os.environ.get("BLAZING_DEVICE_MEM_CONSUMPTION_THRESHOLD", 0.6)
config_options['BLAZ_HOST_MEM_CONSUMPTION_THRESHOLD'] = os.environ.get("BLAZ_HOST_MEM_CONSUMPTION_THRESHOLD", 0.6)
config_options['MAX_KERNEL_RUN_THREADS'] = os.environ.get("MAX_KERNEL_RUN_THREADS", 3)
config_options['TABLE_SCAN_KERNEL_NUM_THREADS'] = os.environ.get("TABLE_SCAN_KERNEL_NUM_THREADS", 1)
config_options['MAX_NUM_ORDER_BY_PARTITIONS_PER_NODE'] = os.environ.get("MAX_NUM_ORDER_BY_PARTITIONS_PER_NODE", 20)
config_options['ORDER_BY_SAMPLES_RATIO'] = os.environ.get("ORDER_BY_SAMPLES_RATIO", 0.0002)
config_options['NUM_BYTES_PER_ORDER_BY_PARTITION'] = os.environ.get("NUM_BYTES_PER_ORDER_BY_PARTITION", 400000000)
config_options['MAX_ORDER_BY_SAMPLES_PER_NODE'] = os.environ.get("MAX_ORDER_BY_SAMPLES_PER_NODE", 10000)
config_options['MAX_SEND_MESSAGE_THREADS'] = os.environ.get("MAX_SEND_MESSAGE_THREADS", 20)
config_options['MEMORY_MONITOR_PERIOD'] = os.environ.get("MEMORY_MONITOR_PERIOD", 50)
config_options['TRANSPORT_BUFFER_BYTE_SIZE'] = os.environ.get("TRANSPORT_BUFFER_BYTE_SIZE", 10485760) # 10 MBs
config_options['TRANSPORT_POOL_NUM_BUFFERS'] = os.environ.get("TRANSPORT_POOL_NUM_BUFFERS", 100)
config_options['BLAZING_LOGGING_DIRECTORY'] = os.environ.get("BSQL_BLAZING_LOGGING_DIRECTORY", 'blazing_log')
config_options['BLAZING_CACHE_DIRECTORY'] = os.environ.get("BSQL_BLAZING_CACHE_DIRECTORY", '/tmp/')
config_options['LOGGING_LEVEL'] = os.environ.get("LOGGING_LEVEL", "trace")
config_options['MAX_JOIN_SCATTER_MEM_OVERHEAD'] = os.environ.get("MAX_JOIN_SCATTER_MEM_OVERHEAD", 500000000)
config_options['NETWORK_INTERFACE'] = os.environ.get("NETWORK_INTERFACE", 'ens5')

# Start the runner

In [ ]:
runner = gpubdb.GPU_BDB_Runner(
    scale='SF1'
    , client_type='cluster'
    , bucket='bsql'
    , data_dir='s3://bsql/data/tpcx_bb/sf1/'
    , output_dir='tpcx-bb-runner/results'
    , **config_options
)

# Use cases for review

## Use case 2

**Question:** Find the top 30 products that are mostly viewed together with a given product in online store. Note that the order of products viewed does not matter, and "viewed together" relates to a web_clickstreams, click_session of a known user with a session timeout of 60 min. If the duration between two click of a user is greater then the session timeout, a new session begins. With a session timeout of 60 min.

Let's peek inside the code:

In [ ]:
q2_code = inspect.getsource(gpubdb.queries.gpu_bdb_queries.gpu_bdb_query_02).split('\n')
print_code('\n'.join(q2_code[92:-18]))

The `get_distinct_sessions` is defined as follows:

In [ ]:
print_code('\n'.join(q2_code[73:77]))

It calls the `get_sessions`

In [ ]:
print_code('\n'.join(q2_code[64:72]))

Let's have a look at the `get_session_id` method

In [ ]:
print_code('\n'.join(q2_code[34:63]))

Now that we know how this works - let's run the query

In [ ]:
runner.run_query(2, repeat=1, validate_results=False)

## Use case 23
**Question:** This Query contains multiple, related iterations: 
1. Iteration 1: Calculate the coefficient of variation and mean of every item and warehouse of the given and the consecutive month. 
2. Iteration 2: Find items that had a coefficient of variation of 1.3 or larger in the given and the consecutive month

In [ ]:
q23_code = inspect.getsource(gpubdb.queries.gpu_bdb_queries.gpu_bdb_query_23).split('\n')
print_code('\n'.join(q23_code[23:-12]))

In [ ]:
runner.run_query(23, repeat=1, validate_results=False)

# Remaining usecases

## Use case 1

**Question:** Find top ***100*** products that are sold together frequently in given stores. Only products in certain categories ***(categories 2 and 3)*** sold in specific stores are considered, and "sold together frequently" means at least ***50*** customers bought these products together in a transaction.

In ANSI-SQL code the solution would look somewhat similar to the one below.

In [ ]:
runner.run_query(1, repeat=1, validate_results=False)

## Use case 3

**Question:** For a given product get a top 30 list sorted by number of views in descending order of the last 5 products that are mostly viewed before the product was purchased online. For the viewed products, consider only products in certain item categories and viewed within 10 days before the purchase date.

In [ ]:
runner.run_query(3, repeat=1, validate_results=False)

## Use case 4

**Question:** Web_clickstream shopping cart abandonment analysis: For users who added products in their shopping carts but did not check out in the online store during their session, find the average number of pages they visited during their sessions. A "session" relates to a click_session of a known user with a session time-out of 60 min. If the duration between two clicks of a user is greater then the session time-out, a new session begins.

In [ ]:
runner.run_query(4, repeat=1, validate_results=False)

## Use case 5

**Question**: Build a model using logistic regression for a visitor to an online store: based on existing users online activities (interest in items of different categories) and demographics. This model will be used to predict if the visitor is interested in a given item category. Output the precision, accuracy and confusion matrix of model. *Note:* no need to actually classify existing users, as it will be later used to predict interests of unknown visitors.

In [ ]:
runner.run_query(5, repeat=1, validate_results=False)

## Use case 6

Identifies customers shifting their purchase habit from store to web sales. Find customers who spend in relation more money in the second year following a given year in the web_sales channel then in the store sales channel. Report customers details: first name, last name, their country of origin, login name and email address, and identify if they are preferred customer, for the top 100 customers with the highest increase intheir second year web purchase ratio.

In [ ]:
runner.run_query(6, repeat=1, validate_results=False)

## Use case 7
**Question:** List top 10 states in descending order with at least 10 customers who during a given month bought products with the price tag at least 20% higher than the average price of products in the same category.

In [ ]:
runner.run_query(7, repeat=1, validate_results=False)

## Use case 8
**Question:** For online sales, compare the total sales monetary amount in which customers checked online reviews before making the purchase and that of sales in which customers did not read reviews. Consider only online sales for a specific category in a given year.

In [ ]:
runner.run_query(8, repeat=1, validate_results=False)

## Use case 9

**Question:** Aggregate total amount of sold items over different given types of combinations of customers based on selected groups of marital status, education status, sales price and different combinations of state and sales/profit.

In [ ]:
runner.run_query(9, repeat=1, validate_results=False)

## Use case 10
**Question:** For all products, extract sentences from its product reviews that contain positive or negative sentiment and display for each item the sentiment polarity of the extracted sentences (POS OR NEG) and the sentence and word in sentence leading to this classification.

In [ ]:
runner.run_query(10, repeat=1, validate_results=False, additional_resources_path='s3://bsql/data/tpcx_bb/additional_resources')

## Use case 11
**Question:** For a given product, measure the correlation of sentiments, including the number of reviews and average review ratings, on product monthly revenues within a given time frame.

In [ ]:
runner.run_query(11, repeat=1, validate_results=False)

## Use case 12
**Question:** Find all customers who viewed items of a given category on the web in a given month and year that was followed by an instore purchase of an item from the same category in the three consecutive months.

In [ ]:
runner.run_query(12, repeat=1, validate_results=False)

## Use case 13
**Question:** Display customers with both store and web sales in consecutive years for whom the increase in web sales exceeds the increase in store sales for a specified year.

In [ ]:
runner.run_query(13, repeat=1, validate_results=False)

## Use case 14
**Question:** What is the ratio between the number of items sold over the internet in the morning (7 to 8am) to the number of items sold in the evening (7 to 8pm) of customers with a specified number of dependents. Consider only websites with a high amount of content.

In [ ]:
runner.run_query(14, repeat=1, validate_results=False)

## Use case 15
**Question:** Find the categories with flat or declining sales for in store purchases during a given year for a given store.

In [ ]:
runner.run_query(15, repeat=1, validate_results=False)

## Use case 16
**Question:** Compute the impact of an item price change on the store sales by computing the total sales for items in a 30 day period before and after the price change. Group the items by location of warehouse where they were delivered from.

In [ ]:
runner.run_query(16, repeat=1, validate_results=False)

## Use case 17
**Question:** Find the ratio of items sold with and without promotions in a given month and year. Only items in certain categories sold to customers living in a specific time zone are considered.

In [ ]:
runner.run_query(17, repeat=1, validate_results=False)

## Use case 18
**Question:** Identify the stores with flat or declining sales in 4 consecutive months, check if there are any negative reviews regarding these stores available online.

In [ ]:
runner.run_query(18, repeat=1, validate_results=False, additional_resources_path='s3://bsql/data/tpcx_bb/additional_resources')

## Use case 19
**Question:** Retrieve the items with the highest number of returns where the number of returns was approximately equivalent across all store and web channels (within a tolerance of +/ 10%), within the week ending given dates. Analyse the online reviews for these items to see if there are any negative reviews.

In [ ]:
runner.run_query(19, repeat=1, validate_results=False, additional_resources_path='s3://bsql/data/tpcx_bb/additional_resources')

## Use case 20
**Question:** Customer segmentation for return analysis: Customers are separated along the following dimensions: 
1. return frequency, 
2. return order ratio (total number of orders partially or fully returned versus the totalnumber of orders), 
3. return item ratio (total number of items returned versus the number of itemspurchased), 
4. return amount ration (total monetary amount of items returned versus the amount purchased),
5. return order ratio. 

Consider the store returns during a given year for the computation.

In [ ]:
runner.run_query(20, repeat=1, validate_results=False)

## Use case 21
**Question:** Get all items that were sold in stores in a given month and year and which were returned in the next 6 months and repurchased by the returning customer afterwards through the web sales channel in the following three years. For those items, compute the total quantity sold through the store, the quantity returned and the quantity purchased through the web. Group this information by item and store.

In [ ]:
runner.run_query(21, repeat=1, validate_results=False)

## Use case 22
**Question:** For all items whose price was changed on a given date, compute the percentage change in inventorybetween the 30 day period BEFORE the price change and the 30 day period AFTER the change. Group this information by warehouse.

In [ ]:
runner.run_query(22, repeat=1, validate_results=False)

## Use case 24
**Question:** For a given product, measure the effect of competitor's prices on products' in store and online sales.Compute the crossprice elasticity of demand for a given product.

In [ ]:
runner.run_query(24, repeat=1, validate_results=False)

## Use case 25
**Question:** Customer segmentation analysis: Customers are separated along the following key shopping dimensions: 
1. recency of last visit, 
2. frequency of visits and monetary amount. 

Use the store and online purchase data during a given year to compute. After model of separation is build, report for the analysed customers towhich "group" they where assigned.

In [ ]:
runner.run_query(25, repeat=1, validate_results=False)

## Use case 26
**Question:** Cluster customers into book buddies/club groups based on their in store book purchasing histories. Aftermodel of separation is build, report for the analysed customers to which "group" they where assigned.

In [ ]:
runner.run_query(26, repeat=1, validate_results=False)

## Use case 27
**Question:** For a given product, find "competitor" company names in the product reviews. Display review id, product id, "competitor’s" company name and the related sentence from the online review

In [ ]:
# runner.run_query(27, repeat=1, validate_results=False)

## Use case 28
**Question:** Build text classifier for online review sentiment classification (Positive, Negative, Neutral), using 90% of available reviews for training and the remaining 10% for testing. Display classifier accuracy on testing dataand classification result for the 10% testing data: \<reviewSK\>, \<originalRating\>, \<classificationResult\>.

In [ ]:
runner.run_query(28, repeat=1, validate_results=False)

## Use case 29
**Question:** Perform category affinity analysis for products purchased together online.

In [ ]:
runner.run_query(29, repeat=1, validate_results=False)

## Use case 30
**Question:** Perform category affinity analysis for products viewed together online. Note that the order of products viewed does not matter, and "viewed together" relates to a click_session of a user with a session timeout of 60 min. If the duration between two clicks of a user is greater then the session timeout, a new session begins.

In [ ]:
runner.run_query(30, repeat=1, validate_results=False)